![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [330]:
# Import necessary packages
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [331]:
#read the data airbnb_price.csv
price = pd.read_csv("data/airbnb_price.csv",index_col='listing_id')
print("first 5 rows of data frame: \n",price.head())
print("\n")
print("*******************************")
print("\n")
print("unique indexes: ",len(price.index.unique()))
print("*******************************")
print("\n")
print("data types are:\n",price.dtypes)

first 5 rows of data frame: 
                   price                nbhood_full
listing_id                                        
2595        225 dollars         Manhattan, Midtown
3831         89 dollars     Brooklyn, Clinton Hill
5099        200 dollars     Manhattan, Murray Hill
5178         79 dollars  Manhattan, Hell's Kitchen
5238        150 dollars       Manhattan, Chinatown


*******************************


unique indexes:  25209
*******************************


data types are:
 price          object
nbhood_full    object
dtype: object


In [328]:
#read the data airbnb_room_type.xlsx
room_type = pd.ExcelFile("data/airbnb_room_type.xlsx")
room_type.sheet_names

room_typ_df = room_type.parse(0)
room_typ_df = room_typ_df.set_index("listing_id")

print("first 5 rows of data frame: \n",room_typ_df.head())
print("\n")
print("*******************************")
print("\n")
print("unique indexes: ",len(room_typ_df.index.unique()))
print("*******************************")
print("\n")
print("data types are:\n",room_typ_df.dtypes)

first 5 rows of data frame: 
                                           description        room_type
listing_id                                                            
2595                            Skylit Midtown Castle  Entire home/apt
3831                  Cozy Entire Floor of Brownstone  Entire home/apt
5099        Large Cozy 1 BR Apartment In Midtown East  Entire home/apt
5178                  Large Furnished Room Near B'way     private room
5238               Cute & Cozy Lower East Side 1 bdrm  Entire home/apt


*******************************


unique indexes:  25209
*******************************


data types are:
 description    object
room_type      object
dtype: object


In [329]:
#read the data airbnb_last_review.tsv
review = pd.read_csv("data/airbnb_last_review.tsv",sep='\t',index_col='listing_id')
review['last_review'] = pd.to_datetime(review['last_review'])
print("first 5 rows of data frame: \n",review.head())
print("\n")
print("*******************************")
print("\n")
print("unique indexes: ",len(review.index.unique()))
print("*******************************")
print("\n")
print("data types are:\n",review.dtypes)

first 5 rows of data frame: 
               host_name last_review
listing_id                         
2595           Jennifer  2019-05-21
3831        LisaRoxanne  2019-07-05
5099              Chris  2019-06-22
5178           Shunichi  2019-06-24
5238                Ben  2019-06-09


*******************************


unique indexes:  25209
*******************************


data types are:
 host_name              object
last_review    datetime64[ns]
dtype: object


In [332]:
#join the dataframes
joined_df = price.join(room_typ_df,on='listing_id',how='inner').join(review,on='listing_id',how='inner')

print("the 5 rows of joined dataframe: ")
print("\n")
print(joined_df.head())

the 5 rows of joined dataframe: 


                  price                nbhood_full  ...    host_name last_review
listing_id                                          ...                         
2595        225 dollars         Manhattan, Midtown  ...     Jennifer  2019-05-21
3831         89 dollars     Brooklyn, Clinton Hill  ...  LisaRoxanne  2019-07-05
5099        200 dollars     Manhattan, Murray Hill  ...        Chris  2019-06-22
5178         79 dollars  Manhattan, Hell's Kitchen  ...     Shunichi  2019-06-24
5238        150 dollars       Manhattan, Chinatown  ...          Ben  2019-06-09

[5 rows x 6 columns]


In [314]:
# What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
min_review_date = joined_df.agg({'last_review':'min'})['last_review']
max_review_date = joined_df.agg({'last_review':'max'})['last_review']

In [315]:
#How many of the listings are private rooms? Save this into any variable.
joined_df['room_type'] = joined_df['room_type'].str.lower()
joined_df_new = joined_df['room_type'].value_counts()
listing_private_rooms = joined_df_new.loc[joined_df_new.index=='private room'][0]
type(listing_private_rooms),listing_private_rooms

(numpy.int64, 11356)

In [316]:
#What is the average listing price? Round to the nearest two decimal places and save into a variable.
joined_df['price'] = joined_df['price'].str.strip(' dollars').astype('int')
avg_listing_price = round(joined_df.agg({'price':'mean'})['price'],2)

In [317]:
# Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.
review_dates = pd.DataFrame({'first_reviewed':[min_review_date],'last_reviewed':[max_review_date],
             'nb_private_rooms':[listing_private_rooms],'avg_price':[avg_listing_price]})
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78
